<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# SQL Together Lab: Learning SQL Syntax

_Authors: Dave Yerrington (SF)_

---

> **This is a hybrid lecture/lab. Each exercise should be completed by a student in front of the class.**


### Learning Objectives
*After this lesson, you will be able to:*
- Sort results by column using `ORDER BY`.
- Simplify your syntax using aliases (`AS`).
- Match patterns using `LIKE`.
- Select distinct items using `DISTINCT`.
- Aggregate values using `GROUP BY`.
- Filter on aggregations using `HAVING`.
- Apply `IF/THEN` logic using `CASE`.
- Use `EXTRACT` to get date parts.

### Lesson Guide
- [Install `psycopg2`](#install-psycopg2)
- [Connect to a Remote Database](#connect-to-remote)
- [Some Notes on Syntax](#syntax-notes)
- [ORDER BY](#order-by)
- [Alias `AS`](#alias-as)
- [LIKE](#like-operator)
- [DISTINCT](#distinct)
- [LIMIT](#limit)
- [GROUP BY](#group-by)
- [HAVING](#having)
- [CASE Statements](#case)
- [Working with Dates](#dates)
- [Additional Exercises](#additional-exercises)
- [Conclusion](#conclusion)
- [Additional Resources](#additional-resources)


<a id='install-psycopg2'></a>
## Install `psycopg2`

---

Either:

`> conda install psycopg2`
> _conda may ask you to update using this method_

Or:

`> pip install psycopg2`



<a id='connect-to-remote'></a>
## Connect to a Remote Database

---

In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

conn_str = "host='dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com' dbname='northwind' user='dsi_student' password='gastudents'"
conn = psycopg2.connect(conn_str)
c = conn.cursor()

<a id='syntax-notes'></a>

## Some Notes on Syntax

---

The [Northwind Database Schema](https://northwinddatabase.codeplex.com/) will come in handy for writing solutions to the problems below. 

1. You should wrap column names in double quotes (**"column_name"**).
2. You can comment out a line by including a double dash in front (**--**).
3. You should wrap a string in single quotes (**'string'**).

```*.sql
SELECT "ProductID" as "PID"
FROM Products
WHERE "ProductName" like '%a' 
--AND 
```

<a id='order-by'></a>

## `ORDER BY`

---

    The `ORDER BY` keyword is used to sort a result set by one or more columns. It sorts records in ascending order by default. To sort the records in descending order, you can use the `DESC` keyword.

### SQL `ORDER BY` Syntax

```*.sql
SELECT column_name1, column_name2  
FROM table_name  
ORDER BY column_name1 ASC, column_name2 DESC;
``` 

### Exercise #1:

Select the `ProductID`, `ProductName`, `SupplierID`, and `UnitPrice` for all `Products` with a `UnitPrice > 25`, ordered by `SupplierID` descending and then `UnitPrice` ascending.

In [2]:
# A:
pd.read_sql('SELECT "ProductID", "ProductName", "SupplierID", "UnitPrice" FROM Products WHERE "UnitPrice" > 25 ORDER BY "SupplierID" DESC, "UnitPrice" ASC;', 
           con = conn).head()


,ProductID,ProductName,SupplierID,UnitPrice
0,61,Sirop d'érable,29,28.5
1,62,Tarte au sucre,29,49.3
2,60,Camembert Pierrot,28,34.0
3,59,Raclette Courdavault,28,55.0
4,56,Gnocchi di nonna Alice,26,38.0


<a id='alias-as'></a>
## Alias `AS`

---

SQL aliases are used to give a database table — or a column in a table — a temporary name. Aliases are often created for two purposes:
1. To make output column names more readable (substitute names). 
2. To make queries more concise (shorten query arguments).

### SQL Alias Syntax for Columns

```*.sql
SELECT column_name AS alias_name  
FROM table_name;
```

### SQL Alias Syntax for Tables

```*.sql
SELECT column_name(s)  
FROM table_name AS alias_name;
```



### Exercise #2

Select `SupplierID` and `CompanyName` from the `Suppliers` table, aliasing these columns as `Supplier No.` and `Company Name`, respectively. Additionally, alias the `Suppliers` table as `S`. Order by `CompanyName` ascending.

In [3]:
# A:
pd.read_sql('SELECT "SupplierID" as "Supplier No.", "CompanyName" as "Company Name" FROM Suppliers as S ORDER BY "Company Name" ASC;', 
           con = conn)

,Supplier No.,Company Name
0,18,Aux joyeux ecclésiastiques
1,16,Bigfoot Breweries
2,5,Cooperativa de Quesos 'Las Cabras'
3,27,Escargots Nouveaux
4,1,Exotic Liquids
5,29,Forêts d'érables
6,14,Formaggi Fortini s.r.l.
7,28,Gai pâturage
8,24,"G'day, Mate"
9,3,Grandma Kelly's Homestead


**Aliases can be useful when:**

- More than one table is involved in a query.
- Functions are used in the query.
- Column names are long and/or not very readable.
- Two or more columns are combined together.

<a id='like-operator'></a>
## SQL's `LIKE` Operator

---

The `LIKE` operator is used in a `WHERE` clause to search for a specific pattern within a column.


### SQL `LIKE` Syntax

```*.sql

SELECT column_name(s) 
FROM table_name  
WHERE column_name LIKE pattern;

```

> **Tip**: The `"%"` sign is used to define wildcards (missing letters) both before and after the pattern. Also, notice that PostgreSQL is case sensitive.

### Exercise #3

In descending order, select all products from the `Products` table with a `ProductName` that contains "ch." Alias this column as `Ch Products`. 

In [4]:
# A:
string = '''
SELECT "ProductName" as "Ch Products" FROM Products WHERE "ProductName" LIKE '%ch%' ORDER BY "Ch Products" DESC;
'''
pd.read_sql(string, con=conn)

,Ch Products
0,Schoggi Schokolade
1,Sasquatch Ale
2,Queso Manchego La Pastora
3,Pâté chinois
4,Gumbär Gummibärchen
5,Gnocchi di nonna Alice


### Exercise #4

In ascending order, select all products from the `Suppliers` table with a `City` that starts with "S." Alias this column as `S Cities`. 

In [5]:
# A:
string = '''
SELECT "City" as "S Cities" FROM Suppliers WHERE "City" LIKE 'S%' ORDER BY "S Cities" ASC;
'''
pd.read_sql(string, con=conn)

,S Cities
0,Salerno
1,Sandvika
2,Sao Paulo
3,Singapore
4,Ste-Hyacinthe
5,Stockholm
6,Sydney


<a id='distinct'></a>
## The `DISTINCT` operator

---

The `SELECT DISTINCT` statement is used to return _only_ distinct (unique) values. In a table, a column may contain many duplicate values; sometimes you'll only want to list the unique ones.

### `SELECT DISTINCT` Syntax

```*.sql

SELECT DISTINCT column_name1, column_name2 
FROM table_name;

```

### Exercise #5

`SELECT DISTINCT` `SupplierID`, `ProductName`, and `UnitPrice` from the `Products` table, ordering by `UnitPrice` ascending (i.e., the cheapest product for each supplier).

In [6]:
# A:
string = '''
SELECT DISTINCT "SupplierID", "ProductName", "UnitPrice" FROM Products ORDER BY "UnitPrice" ASC;
'''
pd.read_sql(string, con=conn).head()

,SupplierID,ProductName,UnitPrice
0,15,Geitost,2.50
1,10,Guaraná Fantástica,4.50
2,6,Konbu,6.00
3,24,Filo Mix,7.00
4,25,Tourtière,7.45


<a id='limit'></a>

## The `LIMIT` operator

---

Sometimes, we may want to only retrieve a fixed number of records from a database. This is where the `LIMIT` operator comes in handy.


### `LIMIT` Syntax

```*.sql

SELECT column_name1, column_name2  
FROM table_name
LIMIT number_of_records;

```

### Exercise #6

In ascending order, return the five highest-priced products that contain an "a" in the product name. Alias the column as `Top 5 A Products`.

In [7]:
# A:
string = '''
SELECT sub."Top 5 A Products", sub."UnitPrice" FROM 
(SELECT "ProductName" as "Top 5 A Products", "UnitPrice" FROM Products 
WHERE "ProductName" LIKE '%a%' ORDER BY "UnitPrice" DESC LIMIT 5) AS sub
ORDER BY "UnitPrice" ASC
'''
pd.read_sql(string, con=conn)

,Top 5 A Products,UnitPrice
0,Raclette Courdavault,55.00
1,Carnarvon Tigers,62.50
2,Sir Rodney's Marmalade,81.00
3,Thüringer Rostbratwurst,123.79
4,Côte de Blaye,263.50


_**Tip:** If you are finding this one a bit tricky to execute in one query, check out [SQL Subqueries](https://www.tutorialspoint.com/sql/sql-sub-queries.htm)._

<a id='group-by'></a>
## `GROUP BY` Operator

---

A table may contain several records that have a common key. 

The `GROUP BY` statement is used in conjunction with aggregate functions to group a result set by one or more columns. For example, we may want to know the total number of items purchased in each order.

### `GROUP BY` Syntax

```*.sql
SELECT column_name, aggregate_function(column_name)  
FROM table_name  
WHERE column_name operator value  
GROUP BY column_name;
```

The aggregate functions you can use with `GROUP BY` are:
- **`COUNT`**
- **`MIN`**
- **`MAX`**
- **`SUM`**
- **`AVG`**

### Exercise #7

From the `Order_details` table, show the count of orders per `OrderID`, as well as the `SUM` of the revenue (`UnitPrice * Quantity`). Order by revenue.

In [8]:
# A:
string = '''
SELECT "OrderID", SUM("UnitPrice"*"Quantity") as "Revenue", COUNT("OrderID") 
FROM Order_details GROUP BY "OrderID" ORDER BY "Revenue" DESC;
'''
#string = '''
#SELECT * FROM Order_details
#'''
pd.read_sql(string, con=conn).head()

,OrderID,Revenue,count
0,10865,17250.000000,2
1,11030,16321.900082,4
2,10981,15810.000000,1
3,10372,12281.200180,4
4,10424,11493.200127,3


<a id='having'></a>
## The `HAVING` operator

---

The `HAVING` clause was added to SQL because the `WHERE` keyword could not be used with aggregate functions. `HAVING` allows us to apply a filter while querying with them. For example, if we only wanted to show companies that had revenues greater than $10,000 (as calculated by an aggregate function).

### `HAVING` Syntax

``` *.sql

SELECT column_name, aggregate_function(column_name)
FROM table_name
WHERE column_name operator value
GROUP BY column_name
HAVING aggregate_function(column_name) operator value;

```

### Exercise #8

Show the revenue of all orders with more than one item.

In [9]:
# A:
# A:
string = '''
SELECT "OrderID", SUM("UnitPrice"*"Quantity") as "Revenue", COUNT("OrderID") 
FROM Order_details GROUP BY "OrderID" 
HAVING COUNT("OrderID") > 1 ORDER BY "Revenue" DESC;
'''
#string = '''
#SELECT * FROM Order_details
#'''
pd.read_sql(string, con=conn).head()

,OrderID,Revenue,count
0,10865,17250.000000,2
1,11030,16321.900082,4
2,10372,12281.200180,4
3,10424,11493.200127,3
4,10817,11490.699940,4


<a id='case'></a>
## `CASE` statements

---

The `CASE` statement is SQL’s way of applying `IF/THEN` logic. The `CASE` statement is followed by at least one pair of `WHEN` and `THEN` statements. It must end with an `END` statement. The `ELSE` statement is optional and provides a way to capture values not specified in the `WHEN/THEN` statements.

### `CASE` Syntax

```*.sql
SELECT 
    CASE WHEN column_name operator value THEN 'string value1'
        WHEN column_name operator value THEN 'string value2'
        ELSE 'string value3' END AS 'alias'         
FROM table_name
```

### A Pseudocode Example

```*.sql
SELECT name
    CASE WHEN age < 1 THEN 'infant'
         WHEN age < 2 THEN 'toddler'
         WHEN age < 5 THEN 'child'
         ELSE 'old as dirt' END AS 'Persons Age'
```

### Exercise #9

Select `CompanyName`, `City`, and `Country` from the `Suppliers` table. Add a new column, `D_F`, which contains a value of "domestic" if the supplier is from the United States and "foreign" if not.

In [10]:
# A:
string = '''
SELECT "CompanyName", "City", "Country",
    CASE 
        WHEN "Country" = 'USA' THEN 'United States'   
        else 'foreign'
        end as "D_F"
         from Suppliers
'''
pd.read_sql(string, con=conn)

,CompanyName,City,Country,D_F
0,Exotic Liquids,London,UK,foreign
1,New Orleans Cajun Delights,New Orleans,USA,United States
2,Grandma Kelly's Homestead,Ann Arbor,USA,United States
3,Tokyo Traders,Tokyo,Japan,foreign
4,Cooperativa de Quesos 'Las Cabras',Oviedo,Spain,foreign
5,Mayumi's,Osaka,Japan,foreign
6,"Pavlova, Ltd.",Melbourne,Australia,foreign
7,"Specialty Biscuits, Ltd.",Manchester,UK,foreign
8,PB Knäckebröd AB,Göteborg,Sweden,foreign
9,Refrescos Americanas LTDA,Sao Paulo,Brazil,foreign


<a id='dates'></a>
## Working With Dates

---

Take some time to look over the [PostgreSQL date documentation](https://www.postgresql.org/docs/8.1/static/functions-datetime.html).

### Extracting Date Parts From a Date Object
```*.sql
SELECT my_date,
       EXTRACT('year'   FROM my_date) AS year,
       EXTRACT('month'  FROM my_date) AS month,
       EXTRACT('day'    FROM my_date) AS day,
       EXTRACT('hour'   FROM my_date) AS hour,
       EXTRACT('minute' FROM my_date) AS minute,
       EXTRACT('second' FROM my_date) AS second,
       EXTRACT('decade' FROM my_date) AS decade,
       EXTRACT('dow'    FROM my_date) AS day_of_week
  FROM table_name
```

### Exercise #10

Select `OrderDate` and `Freight` from the `Orders` table, along with three new columns for `Year`, `Month`, and `Day`. Make sure these are [_**cast**_ as integers, not floats](http://www.postgresqltutorial.com/postgresql-cast/).

After extracting the dates as integers, pull out the `Year`, `Month`, and `SUM` of `Freight`, aliased as `FreightPerMonth`, grouping by the year and month, but only where the freight per month is greater than 5,000.

Order this DataFrame by year and month descending.

In [11]:
# A:
string = '''
SELECT sub."year", sub."month", SUM(sub."Freight") as "FreightPerMonth" FROM
        (SELECT "OrderDate", "Freight", 
            CAST(EXTRACT('year' FROM "OrderDate") AS INTEGER) AS "year",
            CAST(EXTRACT('month' FROM "OrderDate") AS INTEGER) AS "month",
            CAST(EXTRACT('day' FROM "OrderDate") AS INTEGER) AS "day"
        FROM Orders) AS sub
        GROUP BY sub."year", sub."month"
        HAVING SUM(sub."Freight") > 5000
        ORDER BY "year" ASC, "month" ASC;
'''
pd.read_sql(string, con = conn)

,year,month,FreightPerMonth
0,1998,1,5463.44
1,1998,3,5379.02
2,1998,4,6393.57


<a id='additional-exercises'></a>

### Exercise #11

From the `Orders` table, find the average number of days it took to ship a package per `ShipCountry`. Only include orders that have a ship date, and only show the top five results.

In [12]:
# A:
string = '''
select *
from orders
'''
pd.read_sql(string, con = conn).head()

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium


In [2]:
def sql(string):
    return pd.read_sql(string, con = conn)

In [6]:
# A:
sql('''
select "ShipCountry", AVG("ShippedDate" - "OrderDate") as days from orders where "ShippedDate" is not null group by "ShipCountry" order by "days" DESC limit 5
''')

,ShipCountry,days
0,Ireland,11.000000
1,Sweden,10.216216
2,Switzerland,9.941176
3,USA,9.554622
4,Argentina,9.285714


In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

conn_str = "host='dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com' dbname='northwind' user='dsi_student' password='gastudents'"
conn = psycopg2.connect(conn_str)
c = conn.cursor()


### Exercise #12

In the `Orders` table, find the top five countries by average freight cost of products shipped in 1998.

In [17]:
# A:
sql('''
select sub."ShipCountry", AVG(sub."Freight") from
    (select "ShipCountry", "Freight" from orders where extract('year' from "ShippedDate") = 1998) as sub
group by "ShipCountry"
order by AVG(sub."Freight") DESC
limit 5
''')

,ShipCountry,avg
0,Ireland,339.422489
1,Austria,199.137275
2,USA,166.906216
3,Denmark,131.617501
4,Portugal,102.470003


### Exercise #13

From the `Employees` table, find the two women who were hired the most recently. Exclude entries where gender is ambiguous.  
_**Tip:** You may want to investigate the "TitleOfCourtesy" column._

In [11]:
# A:
_ = [print(i[0] + ' ' + i[1]) for i in sql('''
select "FirstName", "LastName" from employees where "TitleOfCourtesy" != 'Dr.' order by "HireDate" ASC limit 2
''').values]

Janet Leverling
Nancy Davolio


In [18]:
# A:

### Exercise #14

Split products from the `Products` table into three price categories:
- **Cheap**: Less than $10.
- **Fair**: $10 to $50.
- **Expensive**: Greater than $50.

Return the count-per-product price categories, along with the `MIN`, `MAX`, and `AVG`. 

In [18]:
# A:
sql('''
select 
CASE WHEN "UnitPrice" < 10 THEN 'Cheap'
    WHEN "UnitPrice" > 50 THEN 'Expensive'
    ELSE 'Fair' end as "price", COUNT("UnitPrice"), min("UnitPrice"), max("UnitPrice"), avg("UnitPrice")
from products
group by "price"
''')

,price,count,min,max,avg
0,Cheap,11,2.5,9.65,7.459091
1,Fair,59,10.0,49.30,23.768983
2,Expensive,7,53.0,263.50,105.112857


<a id='conclusion'></a>
## Conclusion

---

In this lesson, we've learned many new commands for making powerful SQL queries.

In particular, we learned how to:

- Sort results by column using `ORDER BY`.
- Simplify our syntax using aliases.
- Match patterns using `LIKE`.
- Select distinct items using `DISTINCT`.
- Aggregate values using `GROUP BY`.
- Filter aggregations using `HAVING`.
- Apply `IF/THEN` logic using `CASE`.
- Use `EXTRACT` to get date parts.

**Can you think of a few more business cases where these capabilities would be useful?**

<a id='additional-resources'></a>
## Additional Resources

---

- [PostgreSQL Documenation](https://www.postgresql.org/docs/)
- [Mode Analytics Tutorial](https://community.modeanalytics.com/sql/tutorial/introduction-to-sql/)